## How to use the StoG class from PyStoG



In [1]:
from pystog.stog import StoG


ModuleNotFoundError: No module named 'pystog'

Now, lets load in some test data for Argon from our test suite. We extract the $Q$ and $S(Q)$ 
along with the $r$ and $g(r) as well as some material-specific information to initialize StoG.

In [ ]:
from tests.materials import Argon
material = Argon()

from tests.utils import \
    get_data_path, load_data, get_index_of_function, REAL_HEADERS, RECIPROCAL_HEADERS
    
data = load_data(material.reciprocal_space_filename)
q = data[:, get_index_of_function("Q", RECIPROCAL_HEADERS)]
sq = data[:, get_index_of_function("S(Q)", RECIPROCAL_HEADERS)]

data = load_data(material.real_space_filename)
r = data[:, get_index_of_function("r", REAL_HEADERS)]
gofr = data[:, get_index_of_function("g(r)", REAL_HEADERS)]

kwargs_for_stog_input = {
    'NumberDensity': material.kwargs['rho'],
    '<b_coh>^2': material.kwargs['<b_coh>^2'],
    '<b_tot^2>': material.kwargs['<b_tot^2>'],
    'FourierFilter': {'Cutoff': fourier_filter_cutoff},
    'OmittedXrangeCorrection': False,
    'Rdelta': r[1] -r[0],
    'Rmin': min(r),
    'Rmax': max(r)
}

kwargs_for_files = {
    'Files': [
        {'Filename': get_data_path(material.reciprocal_space_filename),
         'ReciprocalFunction': 'S(Q)',
         'Qmin': 0.02,
         'Qmax': 15.0,
         'Y': {'Offset': 0.0,
               'Scale': 1.0},
         'X': {'Offset': 0.0}
         },
        {'Filename': get_data_path(material.reciprocal_space_filename),
         'ReciprocalFunction': 'S(Q)',
         'Qmin': 1.90,
         'Qmax': 35.2,
         'Y': {'Offset': 0.0,
               'Scale': 1.0},
         'X': {'Offset': 0.0}
         }
    ]
}


In [ ]:
stog = StoG(**kwargs_for_stog_input)
stog.plot_flag = False
stog.stem_name = "dog"
stog.files = kwargs_for_files['Files']
stog.read_all_data()
stog.merge_data()
stog.transform_merged()